In [2]:
import pandas as pd
df = pd.read_csv("Retail_Sample.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   LOAN_ID                20 non-null     int64 
 1   REGION_NAME            20 non-null     object
 2   PRINCIPAL_TOTAL        20 non-null     int64 
 3   DISBURSEMENT_DATE      20 non-null     object
 4   PRINCIPAL_OUTSTANDING  20 non-null     int64 
 5   ZONE_ID                20 non-null     object
 6   SALE_AGENT             20 non-null     object
 7   SOCIAL_SCORE           20 non-null     int64 
 8   State                  20 non-null     object
 9   Branch                 20 non-null     object
 10  INDUSTRY               20 non-null     object
dtypes: int64(4), object(7)
memory usage: 1.8+ KB


In [3]:
df.head(2)

,LOAN_ID,REGION_NAME,PRINCIPAL_TOTAL,DISBURSEMENT_DATE,PRINCIPAL_OUTSTANDING,ZONE_ID,SALE_AGENT,SOCIAL_SCORE,State,Branch,INDUSTRY
0,1,GUJARAT,314175,31-07-20,10260,WEST,90053,5,GUJARAT,AHMEDABAD,DAIRY
1,2,PUNJAB,364390,31-07-20,22864,NORTH,90150,6,PUNJAB,LUDHIANA,KIRANA


In [11]:
# Define pivot components
rows_list = ['ZONE_ID','REGION_NAME','State']         # Rows to group by
columns_list = ['INDUSTRY']           # Columns to pivot
filters_list = {}#{'SALE_AGENT':['90053','90150','AG11100','AG11101', '90125']}    
values_list = {'PRINCIPAL_TOTAL','PRINCIPAL_OUTSTANDING'}# Filters (e.g., {'Value1': 30})
aggfunc_list = {'PRINCIPAL_TOTAL': ['sum'],'PRINCIPAL_OUTSTANDING':['sum']}  # Aggregation functions

In [12]:
if filters_list:
    for col, val in filters_list.items():
        df = df[df[col].apply(lambda x: any(item in x for item in val))]

grouped = df.groupby(rows_list + columns_list).agg(aggfunc_list)
pivot = grouped.unstack(columns_list, fill_value=0)

In [13]:
pivot

PRINCIPAL_TOTAL                    \
                                                  sum                     
INDUSTRY                                        DAIRY   KIRANA   OTHERS   
ZONE_ID REGION_NAME    State                                              
EAST    WEST_BENGAL    WEST_BENGAL                  0   362145        0   
NORTH   HARYANA        DELHI/NCR                    0        0   315144   
                       HARYANA                      0  1072982  1255000   
        PUNJAB         PUNJAB                       0   364390        0   
WEST    GUJARAT        GUJARAT                1422607   366753        0   
        MADHYA_PRADESH MADHYA_PRADESH         1090615        0        0   
        MAHARASHTRA    MAHARASHTRA             330000        0   391636   

                                                PRINCIPAL_OUTSTANDING          \
                                                                  sum           
INDUSTRY                              TAILORING                 DAIRY  KIRANA   
ZONE_ID REGION_NAME    State                                                    
EAST    WEST_BENGAL    WEST_BENGAL            0                     0   34366   
NORTH   HARYANA        DELHI/NCR              0                     0       0   
                       HARYANA           212000                     0  885881   
        PUNJAB         PUNJAB                 0                     0   22864   
WEST    GUJARAT        GUJARAT                0                 57623   78048   
        MADHYA_PRADESH MADHYA_PRADESH         0                441547       0   
        MAHARASHTRA    MAHARASHTRA            0                 10869       0   

                                                         
                                                         
INDUSTRY                               OTHERS TAILORING  
ZONE_ID REGION_NAME    State                             
EAST    WEST_BENGAL    WEST_BENGAL          0         0  
NORTH   HARYANA        DELHI/NCR        62961         0  
                       HARYANA         607987     98560  
        PUNJAB         PUNJAB               0         0  
WEST    GUJARAT        GUJARAT              0         0  
        MADHYA_PRADESH MADHYA_PRADESH       0         0  
        MAHARASHTRA    MAHARASHTRA     241945         0

In [18]:
## col wise total...
if len(columns_list) > 0:
    for total in values_list:
        new_df = pivot[total]
        pivot['Total of ' + total] = new_df.sum(axis=1)
        
# row wise total
if len(rows_list)>1:
    new_row = []
    for col in pivot.columns:
        new_row.append(pivot[col].sum())
    new_row_df = pd.DataFrame([new_row], columns=pivot.columns)
    result = pd.concat([pivot, new_row_df])
    


In [19]:
result

PRINCIPAL_TOTAL                    \
                                                   sum                     
INDUSTRY                                         DAIRY   KIRANA   OTHERS   
(EAST, WEST_BENGAL, WEST_BENGAL)                     0   362145        0   
(NORTH, HARYANA, DELHI/NCR)                          0        0   315144   
(NORTH, HARYANA, HARYANA)                            0  1072982  1255000   
(NORTH, PUNJAB, PUNJAB)                              0   364390        0   
(WEST, GUJARAT, GUJARAT)                       1422607   366753        0   
(WEST, MADHYA_PRADESH, MADHYA_PRADESH)         1090615        0        0   
(WEST, MAHARASHTRA, MAHARASHTRA)                330000        0   391636   
0                                              2843222  2166270  1961780   

                                                 PRINCIPAL_OUTSTANDING  \
                                                                   sum   
INDUSTRY                               TAILORING                 DAIRY   
(EAST, WEST_BENGAL, WEST_BENGAL)               0                     0   
(NORTH, HARYANA, DELHI/NCR)                    0                     0   
(NORTH, HARYANA, HARYANA)                 212000                     0   
(NORTH, PUNJAB, PUNJAB)                        0                     0   
(WEST, GUJARAT, GUJARAT)                       0                 57623   
(WEST, MADHYA_PRADESH, MADHYA_PRADESH)         0                441547   
(WEST, MAHARASHTRA, MAHARASHTRA)               0                 10869   
0                                         212000                510039   

                                                                   \
                                                                    
INDUSTRY                                 KIRANA  OTHERS TAILORING   
(EAST, WEST_BENGAL, WEST_BENGAL)          34366       0         0   
(NORTH, HARYANA, DELHI/NCR)                   0   62961         0   
(NORTH, HARYANA, HARYANA)                885881  607987     98560   
(NORTH, PUNJAB, PUNJAB)                   22864       0         0   
(WEST, GUJARAT, GUJARAT)                  78048       0         0   
(WEST, MADHYA_PRADESH, MADHYA_PRADESH)        0       0         0   
(WEST, MAHARASHTRA, MAHARASHTRA)              0  241945         0   
0                                       1021159  912893     98560   

                                       Total of PRINCIPAL_TOTAL  \
                                                                  
INDUSTRY                                                          
(EAST, WEST_BENGAL, WEST_BENGAL)                         362145   
(NORTH, HARYANA, DELHI/NCR)                              315144   
(NORTH, HARYANA, HARYANA)                               2539982   
(NORTH, PUNJAB, PUNJAB)                                  364390   
(WEST, GUJARAT, GUJARAT)                                1789360   
(WEST, MADHYA_PRADESH, MADHYA_PRADESH)                  1090615   
(WEST, MAHARASHTRA, MAHARASHTRA)                         721636   
0                                                       7183272   

                                       Total of PRINCIPAL_OUTSTANDING  
                                                                       
INDUSTRY                                                               
(EAST, WEST_BENGAL, WEST_BENGAL)                                34366  
(NORTH, HARYANA, DELHI/NCR)                                     62961  
(NORTH, HARYANA, HARYANA)                                     1592428  
(NORTH, PUNJAB, PUNJAB)                                         22864  
(WEST, GUJARAT, GUJARAT)                                       135671  
(WEST, MADHYA_PRADESH, MADHYA_PRADESH)                         441547  
(WEST, MAHARASHTRA, MAHARASHTRA)                               252814  
0                                                             2542651

In [21]:
final = pd.DataFrame()
n = len(pivot.index[0])
idx = pd.IndexSlice
for i in range(n):# size of index
    lst = list(set(pivot.index.levels[i])) #ith level of index
    print(f"level--------{i}")
    print(lst)
    for lst_ele in lst:# values of indexlevel
        new_row = []
        arr = [slice(None) if ind != i else lst_ele for ind in range(n)]
        # print(arr)
        idx = pd.IndexSlice[tuple(arr)]
        child_df = pivot.loc[idx, :]
        # print(child_df)
        # agg for ith level of index...
        for col in child_df.columns:
            new_row.append(child_df[col].sum())
        new_row_df = pd.DataFrame([new_row], columns=child_df.columns, index=[lst_ele])
        child_df_new = pd.concat([new_row_df, child_df])
        final = pd.concat([final, child_df_new])
        # print(child_df_new)
    # print("********************************")

level--------0
['EAST', 'NORTH', 'WEST']
level--------1
['MADHYA_PRADESH', 'PUNJAB', 'WEST_BENGAL', 'MAHARASHTRA', 'GUJARAT', 'HARYANA']
level--------2
['MADHYA_PRADESH', 'PUNJAB', 'WEST_BENGAL', 'MAHARASHTRA', 'DELHI/NCR', 'GUJARAT', 'HARYANA']


In [22]:
final

PRINCIPAL_TOTAL                    \
                                                   sum                     
INDUSTRY                                         DAIRY   KIRANA   OTHERS   
EAST                                                 0   362145        0   
(EAST, WEST_BENGAL, WEST_BENGAL)                     0   362145        0   
NORTH                                                0  1437372  1570144   
(NORTH, HARYANA, DELHI/NCR)                          0        0   315144   
(NORTH, HARYANA, HARYANA)                            0  1072982  1255000   
(NORTH, PUNJAB, PUNJAB)                              0   364390        0   
WEST                                           2843222   366753   391636   
(WEST, GUJARAT, GUJARAT)                       1422607   366753        0   
(WEST, MADHYA_PRADESH, MADHYA_PRADESH)         1090615        0        0   
(WEST, MAHARASHTRA, MAHARASHTRA)                330000        0   391636   
MADHYA_PRADESH                                 1090615        0        0   
(WEST, MADHYA_PRADESH, MADHYA_PRADESH)         1090615        0        0   
PUNJAB                                               0   364390        0   
(NORTH, PUNJAB, PUNJAB)                              0   364390        0   
WEST_BENGAL                                          0   362145        0   
(EAST, WEST_BENGAL, WEST_BENGAL)                     0   362145        0   
MAHARASHTRA                                     330000        0   391636   
(WEST, MAHARASHTRA, MAHARASHTRA)                330000        0   391636   
GUJARAT                                        1422607   366753        0   
(WEST, GUJARAT, GUJARAT)                       1422607   366753        0   
HARYANA                                              0  1072982  1570144   
(NORTH, HARYANA, DELHI/NCR)                          0        0   315144   
(NORTH, HARYANA, HARYANA)                            0  1072982  1255000   
MADHYA_PRADESH                                 1090615        0        0   
(WEST, MADHYA_PRADESH, MADHYA_PRADESH)         1090615        0        0   
PUNJAB                                               0   364390        0   
(NORTH, PUNJAB, PUNJAB)                              0   364390        0   
WEST_BENGAL                                          0   362145        0   
(EAST, WEST_BENGAL, WEST_BENGAL)                     0   362145        0   
MAHARASHTRA                                     330000        0   391636   
(WEST, MAHARASHTRA, MAHARASHTRA)                330000        0   391636   
DELHI/NCR                                            0        0   315144   
(NORTH, HARYANA, DELHI/NCR)                          0        0   315144   
GUJARAT                                        1422607   366753        0   
(WEST, GUJARAT, GUJARAT)                       1422607   366753        0   
HARYANA                                              0  1072982  1255000   
(NORTH, HARYANA, HARYANA)                            0  1072982  1255000   

                                                 PRINCIPAL_OUTSTANDING  \
                                                                   sum   
INDUSTRY                               TAILORING                 DAIRY   
EAST                                           0                     0   
(EAST, WEST_BENGAL, WEST_BENGAL)               0                     0   
NORTH                                     212000                     0   
(NORTH, HARYANA, DELHI/NCR)                    0                     0   
(NORTH, HARYANA, HARYANA)                 212000                     0   
(NORTH, PUNJAB, PUNJAB)                        0                     0   
WEST                                           0                510039   
(WEST, GUJARAT, GUJARAT)                       0                 57623   
(WEST, MADHYA_PRADESH, MADHYA_PRADESH)         0                441547   
(WEST, MAHARASHTRA, MAHARASHTRA)               0                 10869   
MADHYA_PRADESH                      